# Library

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import re # regular expressions
import string

# Methods

In [ ]:
def load_stop_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stop_words = set(file.read().splitlines())
    return stop_words

# Main

## Dataset

In [ ]:
dataset = pd.read_json("dataset\Turkish_data_depository_630koseyazisi.jsonl",lines=True)
print(dataset.head(10))
print(dataset.info())

In [ ]:
x = dataset.iloc[:, 2].values.astype("str") # author text
y = dataset.iloc[:, 0].values.astype("str") # author name

print(f"type x : {type(x)} , type y : {type(y)}")

## Preprocesing

In [ ]:
# turkish stop words
stop_words = load_stop_words("stopwordsTR\stopwords.txt")

# Her bir metni temizleme işlemi
for i in range(len(x)):
    # Convert to lowercase - Küçük harfe çevirme
    x[i] = x[i].lower()
    
    # Remove escape characters - Kaçış karakterlerini kaldırma
    x[i] = re.sub(r'[\r\n]', ' ', x[i]) 
    
    # Remove unnecessary characters - Gereksiz karakterleri kaldırma
    x[i] = re.sub(r'[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', '', x[i])
    
    # Remove stop words - Stop kelimeleri kaldırma
    x[i] = ' '.join([word for word in x[i].split() if word not in stop_words])
    
    # Remove punctuation marks - Noktalama işaretlerini kaldırma
    x[i] = x[i].translate(str.maketrans('', '', string.punctuation))
    
    # Remove repeating spaces - Tekrar eden boşlukları kaldırma
    x[i] = re.sub(r'\s+', ' ', x[i])
    

# Convert to feature vector

## TF-IDF (Term Frequency-Inverse Document Frequency):
It is a statistical method used to measure the importance of words in text documents. TF-IDF vectors combine each word's frequency (TF) in the text and its rarity (IDF) across all documents.

Metin belgelerindeki kelimelerin önemini ölçmek için kullanılan bir istatistiksel yöntemdir. TF-IDF vektörleri, her kelimenin metindeki sıklığını (TF) ve tüm belgelerdeki nadirliğini (IDF) birleştirir.

In [ ]:
# Create TF-IDF vector - TF-IDF vektörlerini oluşturma
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(x)

example_tfidf_vector = tfidf_matrix[0]
print("TF-IDF Vektörü (Örnek):", example_tfidf_vector)

## Word Embeddings (Kelime Gömme):
It represents words with numerical vectors using word embedding models with techniques such as Word2Vec, GloVe, FastText. These vectors better reflect the meaning of the word.

Word2Vec, GloVe, FastText gibi tekniklerle kelime gömme modelleri kullanarak kelimeleri sayısal vektörlerle temsil eder. Bu vektörler, kelimenin anlamını daha iyi yansıtır.

In [ ]:
# Create BoW vector - BoW vektörlerini oluşturma
bow_vectorizer = CountVectorizer()
bow_matrix = bow_vectorizer.fit_transform(x)

example_bow_vector = bow_matrix[0]
print("BoW Vektörü (Örnek):", example_bow_vector)

## N-gram Approach(N-gram Yaklaşımı):
It creates feature vectors by taking into account consecutive word groups of the text.

Metnin ardışık kelime gruplarını dikkate alarak özellik vektörleri oluşturur. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create Trigram BoW vector - Trigram BoW vektörlerini oluşturma
trigram_vectorizer = CountVectorizer(ngram_range=(3,3))
trigram_matrix = trigram_vectorizer.fit_transform(x)

example_trigram_vector = trigram_matrix[0]
print("Trigram BoW Vektörü (Örnek):", example_trigram_vector)

# Machine Learning Models 

## BoW (Bag of Words) Vektörleri ile Karar Ağacı (Decision Tree) Modeli:

In [ ]:
# Verileri eğitim ve test setlerine ayırma
x_train, x_test, y_train, y_test = train_test_split(bow_matrix, y, test_size=0.2, random_state=42)

# Karar Ağacı modelini oluşturma ve eğitme
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)

# Test seti üzerinde modelin performansını değerlendirme
y_pred = decision_tree.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Karar Ağacı Modeli Doğruluğu:", accuracy)

# Karar Ağacı modelinin performansını görselleştirme
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()

# Sınıflandırma raporunu görüntüleme
class_report = classification_report(y_test, y_pred)
print(class_report)



## TF-IDF Vektörleri ile Destek Vektör Makinesi (Support Vector Machine - SVM) Modeli:

In [ ]:
# Verileri eğitim ve test setlerine ayırma
x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=42)

# Destek Vektör Makinesi (SVM) modelini oluşturma ve eğitme
svm_model = SVC(kernel='linear')
svm_model.fit(x_train, y_train)

# Test seti üzerinde modelin performansını değerlendirme
y_pred = svm_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("SVM Modeli Doğruluğu:", accuracy)

# karar ağacı modelinin performansını görselleştirme
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()

# Sınıflandırma raporunu görüntüleme
class_report = classification_report(y_test, y_pred)
print(class_report)


## N-gram BoW Vektörleri ile Naive Bayes Modeli:

In [ ]:

# Verileri eğitim ve test setlerine ayırma
x_train, x_test, y_train, y_test = train_test_split(trigram_matrix, y, test_size=0.2, random_state=42)

# Naive Bayes modelini oluşturma ve eğitme
naive_bayes = MultinomialNB()
naive_bayes.fit(x_train, y_train)

# Test seti üzerinde modelin performansını değerlendirme
y_pred = naive_bayes.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Modeli Doğruluğu:", accuracy)

# naive bayes modelinin performansını görselleştirme
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()

# Sınıflandırma raporunu görüntüleme
class_report = classification_report(y_test, y_pred)
print(class_report)


## Random Forest:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(bow_matrix, y, test_size=0.2, random_state=42)

rnd_forest = RandomForestClassifier()
rnd_forest.fit(x_train, y_train)

y_pred = rnd_forest.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Modeli Doğruluğu:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()

classification_report = classification_report(y_test, y_pred)
print(classification_report)


## K-Nearest Neighbors (KNN):

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(bow_matrix, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("KNN Modeli Doğruluğu:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()


## Gradient Boosting:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(bow_matrix, y, test_size=0.2, random_state=42)

gradient_boosting = GradientBoostingClassifier()
gradient_boosting.fit(x_train, y_train)

y_pred = gradient_boosting.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Gradient Boosting Modeli Doğruluğu:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()
